In [1]:
import os

# Set the environment variable
os.environ['SCIPY_USE_PROPACK'] = '1'

import numpy as np
import matplotlib.pyplot as plt
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/bpauldub/ShapleyFolkman/')
sys.path.insert(1, '/home/bpauldub/ShapleyFolkman/code/')

from frank_wolfe_1 import *
from frank_wolfe_1_strongly_convex import *
from sparsify_constructive import *
from sparsify_frank_wolfe import frank_wolfe_2_fully_corrective, compute_f_values, build_final_solution_fcfw
from randomized_solver import solve_boyd


import scipy as sp
import os
from utils import *
from solve_dual import *

/home/bpauldub/miniconda3/envs/shapley/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [108]:
n = 50
m = 10
random_seed = 2
rho = 3

prob = create_toy_problem(n, m, rho=rho, random_seed=random_seed)
prob.A =  prob.A
x_star = solve_bidual(prob,verbose=False)
opt_dual_value = prob.f_biconjugate(x_star)
x_boyd = solve_boyd(prob, opt_dual_value)
for i in range(n):
    if np.abs(x_boyd[i] - 0.4) < 1e-3:
        x_boyd[i] = 0.4
    if np.abs(x_boyd[i] - 0.6) < 1e-3:
        x_boyd[i] = 0.6


opt_dual_value, prob.f(x_star), prob.f_biconjugate(x_boyd), prob.f(x_boyd)

(50, 500, 50.287670490460705, 131.2876704904607)

In [109]:
x_star, x_boyd, prob.A @ x_boyd - prob.b

(array([0.50295249, 0.43541305, 0.4214435 , 0.53372025, 0.4277401 ,
        0.5492066 , 0.55575889, 0.53456533, 0.51354834, 0.41920348,
        0.55061389, 0.53433924, 0.58934775, 0.57369028, 0.54609403,
        0.49024116, 0.46812153, 0.46135135, 0.4       , 0.563272  ,
        0.48527324, 0.48167992, 0.47823791, 0.59878006, 0.42977261,
        0.51590069, 0.53471542, 0.58735639, 0.44684133, 0.42497112,
        0.43646285, 0.52310993, 0.53648238, 0.41277262, 0.53673806,
        0.41678072, 0.52526391, 0.55947263, 0.54849031, 0.42996133,
        0.41521334, 0.43311242, 0.56381955, 0.40018148, 0.46546117,
        0.42910333, 0.46960797, 0.43671808, 0.43602842, 0.4257988 ]),
 array([0.40815521, 0.6       , 0.4       , 0.4       , 0.6       ,
        0.6       , 0.6       , 0.4       , 0.6       , 0.61101888,
        0.4       , 0.6       , 0.4       , 0.6       , 0.6       ,
        0.6       , 0.6       , 0.6       , 0.45573422, 0.6       ,
        0.4       , 0.6       , 0.6       , 0.

In [110]:
x_start = np.zeros((1, n))
K = 10000
z_K, grad_norm_list, y_dic, = frank_wolfe_1(opt_dual_value, prob.b, K, prob, x_start)

Iteration 100 : ||z_k - z*||_+^2 = 4.953203119068824 + 0.0
     ||grad(z_k)|| = 2.22557927719253
Iteration 200 : ||z_k - z*||_+^2 = 2.4201690508797955 + 0.00023936817602606555
     ||grad(z_k)|| = 1.5557661839286203
Iteration 300 : ||z_k - z*||_+^2 = 1.6148606999639434 + 0.00014848807347970694
     ||grad(z_k)|| = 1.2708301176937156
Iteration 400 : ||z_k - z*||_+^2 = 1.2140585501268664 + 0.0
     ||grad(z_k)|| = 1.1018432511600125
Iteration 500 : ||z_k - z*||_+^2 = 0.9673417884318479 + 3.3521229856624622e-06
     ||grad(z_k)|| = 0.9835370560150917
Iteration 600 : ||z_k - z*||_+^2 = 0.8036322246553464 + 0.0004384914660147337
     ||grad(z_k)|| = 0.8966999030452503
Iteration 700 : ||z_k - z*||_+^2 = 0.6901084301456288 + 1.5004835962293843e-05
     ||grad(z_k)|| = 0.8307366820970356
Iteration 800 : ||z_k - z*||_+^2 = 0.6059863120192699 + 0.00025870755790627977
     ||grad(z_k)|| = 0.7786173768785128
Iteration 900 : ||z_k - z*||_+^2 = 0.5367644842381611 + 2.1122525290007574e-06
     ||grad

In [111]:
Z_matrix, eta_vector, corresponding_indices = create_Z_matrix(y_dic, prob)
eta_vector[eta_vector < 0] = 0
print((Z_matrix @ eta_vector)[:m+1] - z_K)

[ 2.62900812e-13  3.12638804e-13  4.26325641e-14 -3.10862447e-13
 -2.13162821e-13 -1.35003120e-13  2.91322522e-13 -2.70006240e-13
  8.52651283e-13  2.34479103e-13 -5.54223334e-13]


In [112]:
final_Z_matrix, final_eta_vector, final_corresponding_indices = sparsify_solution_caratheodory(Z_matrix, eta_vector, corresponding_indices)

0 (62, 62) 0.0 (151,)


/home/bpauldub/ShapleyFolkman/code/sparsify_constructive.py:89: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  delta = np.linalg.lstsq(R, Q.T @ rhs)[0]
/home/bpauldub/ShapleyFolkman/code/sparsify_constructive.py:100: RuntimeWarning: divide by zero encountered in divide
  candidates = eta_vector_copy[:Z_matrix.shape[0] + 1] / delta


In [113]:
y_final, y_final_max, y_final_sampled = build_final_solution_caratheodory(y_dic, final_eta_vector, final_corresponding_indices)
y_final[y_final > 1 - 1e-4] = 1
y_final_col = y_final.reshape((-1,), order='F')
y_final_col_max = y_final_max.reshape((-1,), order='F')
y_final_col_sampled = y_final_sampled.reshape((-1,), order='F')
for i in range(n):
    if np.abs(y_final[0, i] - 0.4) < 1e-3:
        y_final[0, i] = 0.4
    if np.abs(y_final[0, i] - 0.6) < 1e-3:
        y_final[0, i] = 0.6

In [114]:
prob.f(y_final_sampled), prob.f(y_final_max), prob.f(y_final),  prob.f(x_star), prob.f_biconjugate(x_star)

(50.015818571779256, 50.015818571779256, 140.0, 500, 50)

In [150]:
for k in range(10000):
    _, _, y_final_sampled = build_final_solution_caratheodory(y_dic, final_eta_vector, final_corresponding_indices)
    y_final_col_sampled = y_final_sampled.reshape((-1,), order='F')
    if np.max(np.clip(prob.A @ y_final_col_sampled - prob.b, 0, None)) <= np.max(prob.A @ y_final_col - prob.b):
        print(prob.f(y_final_sampled), np.max(np.clip(prob.A @ y_final_col_sampled - prob.b, 0, None)))

50.015818571779256 0.0
50.015818571779256 0.0
50.015818571779256 0.0


In [116]:
print(np.max(prob.A @ y_final_col - prob.b))

0.020346607338950662


In [117]:
y_final

array([[0.6       , 0.4       , 0.6       , 0.4       , 0.4       ,
        0.4       , 0.6       , 0.4       , 0.4       , 0.6       ,
        0.6       , 0.6       , 0.6       , 0.6       , 0.4       ,
        0.6       , 0.6       , 0.4       , 0.6       , 0.49971533,
        0.6       , 0.6       , 0.4       , 0.6       , 0.6       ,
        0.46838937, 0.6       , 0.4       , 0.4       , 0.4       ,
        0.4       , 0.6       , 0.6       , 0.6       , 0.6       ,
        0.4       , 0.48450061, 0.5306447 , 0.5056275 , 0.4       ,
        0.55795357, 0.4       , 0.6       , 0.59637562, 0.56344796,
        0.6       , 0.6       , 0.51477008, 0.4       , 0.51528656]])

In [118]:
prob.f(y_final)

140.0

In [164]:
T = n + 30
z_t, active_set, grad_list, opt_lambda, V_matrix, _ = frank_wolfe_2_fully_corrective(z_K, T, y_dic, prob, f_values=None)

10 0
Adding index (29, 1) to active set
[0.50243903]
At iteration 1, grad_norm = 24.210108484919466, min(opt_lambda) = 0.4975607095139165, nb deleted = 0, nb total deleted = 0
In the final sum: missing indices = 48, trivial CC = 2, non-trivial CC = 0
Adding index (5, 1) to active set
[0.44160389]
At iteration 2, grad_norm = 16.629546891828817, min(opt_lambda) = 0.1221564484198722, nb deleted = 0, nb total deleted = 0
In the final sum: missing indices = 47, trivial CC = 3, non-trivial CC = 0
Adding index (31, 1) to active set
[0.18495404]
At iteration 3, grad_norm = 7.606102030979111, min(opt_lambda) = 0.07255195509679804, nb deleted = 0, nb total deleted = 0
In the final sum: missing indices = 46, trivial CC = 4, non-trivial CC = 0
Adding index (10, 1) to active set
[0.04020394 0.19206646]
At iteration 4, grad_norm = 5.716859562169083, min(opt_lambda) = 0.04020393708708557, nb deleted = 0, nb total deleted = 0
In the final sum: missing indices = 46, trivial CC = 3, non-trivial CC = 1
A

In [165]:
y_final_fcfw, y_final_fcfw_max = build_final_solution_fcfw(y_dic, opt_lambda, active_set)
y_final_fcfw_col = y_final_fcfw.reshape((-1,), order='F')
y_final_fcfw_col_max = y_final_fcfw_max.reshape((-1,), order='F')
for i in range(n):
    if np.abs(y_final_fcfw[0, i] - 0.4) < 1e-3:
        y_final_fcfw[0, i] = 0.4
    if np.abs(y_final_fcfw[0, i] - 0.6) < 1e-3:
        y_final_fcfw[0, i] = 0.6

In [166]:
prob.f(y_final), prob.f(y_final_fcfw),  prob.f(x_star), prob.f_biconjugate(x_star)

(140.0, 131.0, 500, 50)

In [167]:
print(np.max(prob.A @ y_final_fcfw_col - prob.b))

0.08756435986784084


In [168]:
y_final_fcfw

array([[0.47114512, 0.6       , 0.49939156, 0.6       , 0.42332554,
        0.6       , 0.6       , 0.6       , 0.6       , 0.6       ,
        0.6       , 0.6       , 0.6       , 0.57656268, 0.6       ,
        0.6       , 0.4       , 0.6       , 0.4       , 0.59493018,
        0.6       , 0.6       , 0.52274961, 0.6       , 0.6       ,
        0.6       , 0.6       , 0.6       , 0.6       , 0.44093116,
        0.6       , 0.6       , 0.6       , 0.6       , 0.6       ,
        0.6       , 0.50006335, 0.6       , 0.6       , 0.6       ,
        0.4       , 0.59713692, 0.6       , 0.6       , 0.4       ,
        0.6       , 0.6       , 0.6       , 0.6       , 0.6       ]])

In [161]:
y_final

array([[0.6       , 0.4       , 0.6       , 0.4       , 0.4       ,
        0.4       , 0.6       , 0.4       , 0.4       , 0.6       ,
        0.6       , 0.6       , 0.6       , 0.6       , 0.4       ,
        0.6       , 0.6       , 0.4       , 0.6       , 0.49971533,
        0.6       , 0.6       , 0.4       , 0.6       , 0.6       ,
        0.46838937, 0.6       , 0.4       , 0.4       , 0.4       ,
        0.4       , 0.6       , 0.6       , 0.6       , 0.6       ,
        0.4       , 0.48450061, 0.5306447 , 0.5056275 , 0.4       ,
        0.55795357, 0.4       , 0.6       , 0.59637562, 0.56344796,
        0.6       , 0.6       , 0.51477008, 0.4       , 0.51528656]])

In [163]:
prob.A @ y_final_col - prob.b

array([-2.11951396, -0.01623387, -0.01407989,  0.00921549, -3.02256251,
       -5.82648368,  0.01118435,  0.02034661, -3.82480552, -7.1008024 ])